In [ ]:
import pandas as pd

PATH = ''

ALARM_VIOLATIONS = pd.read_csv(PATH+'alarm_violations.csv')

ALARM_VIOLATIONS.head()



In [ ]:
ALARM_VIOLATIONS.head()


In [ ]:
ALARM_VIOLATIONS['ITEMID']= ALARM_VIOLATIONS['ITEMID'].astype(str)
ALARM_VIOLATIONS['ICUSTAY_ID']= ALARM_VIOLATIONS['ICUSTAY_ID'].astype(str)
ALARM_VIOLATIONS.describe()

### Wie viele unique ICU stays befinden sich in dem datenset?
Es befinden sich 19 968 unique ICU Aufenthalte in dem datenset

In [ ]:

unique_ICU_stays = ALARM_VIOLATIONS["ICUSTAY_ID"].value_counts()
len(unique_ICU_stays)


### Wie häufig wird ein alarm ausgelöst? = row count alarm violations
Es befinden sich 388 209 ausgelöste alarme in dem datenset

In [ ]:
len(ALARM_VIOLATIONS)


# Stratify Alarms by ITEMID and TYPE (HIGH/LOW)

### Werden Alarme eher durch unterschreiten oder durch überschreiten der threshholds ausgelöst?

In [ ]:
ALARM_VIOLATIONS_HIGH = ALARM_VIOLATIONS[(ALARM_VIOLATIONS['THRESHOLD_TYPE'] == 'HIGH')]
print("High ALarms:",len(ALARM_VIOLATIONS_HIGH))

ALARM_VIOLATIONS_LOW = ALARM_VIOLATIONS[(ALARM_VIOLATIONS['THRESHOLD_TYPE'] == 'LOW')]
print("Low ALarms:",len(ALARM_VIOLATIONS_LOW))


### Aufgeteilt nach ITEMD ID

In [ ]:
ALARM_VIOLATIONS_STRATFIED = ALARM_VIOLATIONS.groupby(['ITEMID','THRESHOLD_TYPE']).size().reset_index(name='count')
print(ALARM_VIOLATIONS_STRATFIED)
ALARM_VIOLATIONS_STRATFIED.dtypes

### Visualisierung in Barcharts


In [ ]:
import numpy as np

ALARM_VIOLATIONS_STRATFIED_T= ALARM_VIOLATIONS_STRATFIED.pivot(index='ITEMID', columns='THRESHOLD_TYPE', values='count')
ALARM_VIOLATIONS_STRATFIED_T


In [ ]:
# define figure
fig, ax = plt.subplots(1, figsize=(16, 6))
# numerical x
x = np.arange(0, len(ALARM_VIOLATIONS_STRATFIED_T.index))
# plot bars
plt.bar(x - 0.3, ALARM_VIOLATIONS_STRATFIED_T['LOW'], width = 0.2, color = '#1D2F6F')
plt.bar(x - 0.1, ALARM_VIOLATIONS_STRATFIED_T['HIGH'], width = 0.2, color = '#8390FA')

# x y details
plt.xlabel('ITEM ID',fontsize=16)
plt.ylabel('Alarm Counts',fontsize=16)
plt.xticks(x, ALARM_VIOLATIONS_STRATFIED_T.index)

# title and legend
plt.title('Alarm Counts by Item Id and Type', fontsize=18)
plt.legend(['LOW','HIGH'], loc='upper left', ncol = 2)

plt.show()


# Average # of Alarm Violations per ICU Stay
 Wie viele alarm violations haben wir pro ICU STAY?


In [ ]:

unique_ICU_stays = ALARM_VIOLATIONS["ICUSTAY_ID"].value_counts()

# convert to data frame
df_unique_ICU_stays = pd.DataFrame(unique_ICU_stays)
df_unique_ICU_stays = df_unique_ICU_stays.reset_index()
df_unique_ICU_stays.columns = ['ICUSTAY_ID','AlarmCount']
df_unique_ICU_stays['ICUSTAY_ID']=df_unique_ICU_stays['ICUSTAY_ID'].str.rstrip('.0')
df_unique_ICU_stays.describe()


In [ ]:
mean_alarms_per_stay = df_unique_ICU_stays['AlarmCount'].mean()
median_alarms_per_stay = df_unique_ICU_stays['AlarmCount'].median()
min_alarms_per_stay = df_unique_ICU_stays['AlarmCount'].min()
max_alarms_per_stay = df_unique_ICU_stays['AlarmCount'].max()
print('Mean Alarms per Stay:',mean_alarms_per_stay)
print('Median Alarms per Stay:',median_alarms_per_stay)
print('Min Alarms per Stay:',min_alarms_per_stay)
print('Max Alarms per Stay:',max_alarms_per_stay)

### Visualisierung in Strip plots

In [ ]:
count_alarm_numbers = df_unique_ICU_stays['AlarmCount'].value_counts()
count_alarm_numbers
# there were 2490 ICU stays with only one alarm 

In [ ]:
import seaborn as sns

sns.set_style("whitegrid")

fig, ax = plt.subplots(1, figsize=(16, 6))

sns.stripplot(data=df_unique_ICU_stays,x='AlarmCount')
plt.title("Alarm Count - Scatter Plot",fontsize=18)
plt.xlabel("Alarm Count",fontsize=16)

plt.show()



### Analyse der Alarm Counts per ICU Aufenthalt nach Item Id

In [ ]:
# Create dataframe
unique_ICU_stays_by_ItemId = ALARM_VIOLATIONS.groupby(['ITEMID','ICUSTAY_ID']).size().reset_index(name='AlarmCount')
unique_ICU_stays_by_ItemId['ICUSTAY_ID']=unique_ICU_stays_by_ItemId['ICUSTAY_ID'].str.rstrip('.0')
unique_ICU_stays_by_ItemId
unique_ICU_stays_by_ItemId.sort_values(by=['AlarmCount'], inplace=True)
unique_ICU_stays_by_ItemId
# ICU Stay Id with highest Alarm Counts for Item Id 220277 equals the one above with ~14 000 alarms

In [ ]:
unique_ICU_stays_by_220045= unique_ICU_stays_by_ItemId[(unique_ICU_stays_by_ItemId["ITEMID"] =="220045")]
unique_ICU_stays_by_220045.describe()

In [ ]:
unique_ICU_stays_by_220179= unique_ICU_stays_by_ItemId[(unique_ICU_stays_by_ItemId["ITEMID"] =="220179")]
unique_ICU_stays_by_220179.describe()

In [ ]:
unique_ICU_stays_by_220210= unique_ICU_stays_by_ItemId[(unique_ICU_stays_by_ItemId["ITEMID"] =="220210")]
unique_ICU_stays_by_220210.describe()

In [ ]:
unique_ICU_stays_by_220277= unique_ICU_stays_by_ItemId[(unique_ICU_stays_by_ItemId["ITEMID"] =="220277")]
unique_ICU_stays_by_220277.describe()

In [ ]:
unique_ICU_stays_by_224687= unique_ICU_stays_by_ItemId[(unique_ICU_stays_by_ItemId["ITEMID"] =="224687")]
unique_ICU_stays_by_224687.describe()

### Visualisierung in Strip plot

In [ ]:
# define figure
fig, ax = plt.subplots(1, figsize=(16, 6))
# numerical x
x = np.arange(0, len(ALARM_VIOLATIONS_STRATFIED_T.index))
# plot bars
plt.bar(x - 0.3, ALARM_VIOLATIONS_STRATFIED_T['LOW'], width = 0.2, color = '#1D2F6F')
plt.bar(x - 0.1, ALARM_VIOLATIONS_STRATFIED_T['HIGH'], width = 0.2, color = '#8390FA')

# x y details
plt.xlabel('ITEM ID')
plt.ylabel('Alarm Counts')
plt.xticks(x, ALARM_VIOLATIONS_STRATFIED_T.index)

# title and legend
plt.title('Alarm Counts by Item Id and Type', loc ='left')
plt.legend(['LOW','HIGH'], loc='upper left', ncol = 2)

plt.show()

In [ ]:
#Stratified by Item Id
#sns.stripplot(x="sports", y="QOL", data=gold, ax=axs[0])
#axs[0].set_title("sports scatter plot")
#axs[0].set_xlabel("sports (higher value means less sport)")
#%%
sns.set_style("whitegrid")

fig, ax = plt.subplots(1, figsize=(16, 6))

sns.stripplot(x="ITEMID", y="AlarmCount", data=unique_ICU_stays_by_ItemId)
plt.title("Alarm Count by Item ID - Scatter Plot",fontsize=18)
plt.xlabel("Item Id",fontsize=16)
plt.ylabel("Alarm Count",fontsize=16)
plt.gca().set_ylim(bottom=0)
plt.show()

In [ ]:
ALARM_VIOLATIONS.head()

# Analysis of Difference between actual values and thresholds

In [ ]:
# create new column that shows the dif between actual and threshold
ALARM_VIOLATIONS['DIF_ACTUAL_TH'] = ALARM_VIOLATIONS['VALUENUM'] - ALARM_VIOLATIONS['THRESHOLD_VALUE']
ALARM_VIOLATIONS.head()



In [ ]:
# analyze dif by item id
dif_analysis = ALARM_VIOLATIONS.groupby('ITEMID').describe()
dif_analysis = dif_analysis["DIF_ACTUAL_TH"].round(2)
dif_analysis

## Analyze dif for each Item ID

### 220045 - Heart Rate 

In [ ]:
ALARM_VIOLATIONS_220045 = ALARM_VIOLATIONS[(ALARM_VIOLATIONS['ITEMID'] == '220045')]
ALARM_VIOLATIONS_220045

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
ALARM_VIOLATIONS_220045 = ALARM_VIOLATIONS[(ALARM_VIOLATIONS['ITEMID'] == '220045')]

fig, axs = plt.subplots(1, 2, figsize=(25, 5))
fig.suptitle("Difference Between Actual and Threshold - 220045", fontsize=18)

sns.stripplot(data=ALARM_VIOLATIONS_220045,x='THRESHOLD_TYPE', y='DIF_ACTUAL_TH', ax=axs[0])
#axs[0].set_title("Difference Between Actual and Threshold - 220045 Scatter Plot",fontsize=14)
axs[0].set_ylabel("Difference Between Actual and Threshold",fontsize=14)
axs[0].set_xlabel("Threshold Type",fontsize=14)

sns.boxplot(data=ALARM_VIOLATIONS_220045,x='THRESHOLD_TYPE', y='DIF_ACTUAL_TH', ax=axs[1])
#axs[1].set_title("Difference Between Actual and Threshold - 220045 Boxplot")
axs[1].set_ylabel("Difference Between Actual and Threshold", fontsize=14)
axs[1].set_xlabel("Threshold Type",fontsize=14)

#sns.histplot(data=ALARM_VIOLATIONS_220045, x='DIF_ACTUAL_TH', ax=axs[2])
#axs[2].set_title("HR_violations_clean histogram")
#axs[2].set_xlabel("HR_violations_clean VALUENUM")

plt.show(fig)


In [ ]:
ALARM_VIOLATIONS_220045.describe()
ALARM_VIOLATIONS_220045_H = ALARM_VIOLATIONS_220045[(ALARM_VIOLATIONS_220045['THRESHOLD_TYPE'] == 'HIGH')]
ALARM_VIOLATIONS_220045_H.describe()

In [ ]:
ALARM_VIOLATIONS_220045_L = ALARM_VIOLATIONS_220045[(ALARM_VIOLATIONS_220045['THRESHOLD_TYPE'] == 'LOW')]
ALARM_VIOLATIONS_220045_L.describe()

### 220179 - Non Invasive Blood Pressure systolic

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
ALARM_VIOLATIONS_220179 = ALARM_VIOLATIONS[(ALARM_VIOLATIONS['ITEMID'] == '220179')]

fig, axs = plt.subplots(1, 2, figsize=(25, 5))
fig.suptitle("Difference Between Actual and Threshold - 220179", fontsize=18)

sns.stripplot(data=ALARM_VIOLATIONS_220179,x='THRESHOLD_TYPE', y='DIF_ACTUAL_TH', ax=axs[0])
#axs[0].set_title("Difference Between Actual and Threshold - 220045 Scatter Plot",fontsize=14)
axs[0].set_ylabel("Difference Between Actual and Threshold",fontsize=14)
axs[0].set_xlabel("Threshold Type",fontsize=14)

sns.boxplot(data=ALARM_VIOLATIONS_220179,x='THRESHOLD_TYPE', y='DIF_ACTUAL_TH', ax=axs[1])
#axs[1].set_title("Difference Between Actual and Threshold - 220045 Boxplot")
axs[1].set_ylabel("Difference Between Actual and Threshold", fontsize=14)
axs[1].set_xlabel("Threshold Type",fontsize=14)

#sns.histplot(data=ALARM_VIOLATIONS_220179, x='DIF_ACTUAL_TH', ax=axs[2])
#axs[2].set_title("HR_violations_clean histogram")
#axs[2].set_xlabel("HR_violations_clean VALUENUM")

plt.show(fig)

### 220210 - Respiratory Rate

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
ALARM_VIOLATIONS_220210 = ALARM_VIOLATIONS[(ALARM_VIOLATIONS['ITEMID'] == '220210')]

fig, axs = plt.subplots(1, 2, figsize=(25, 5))
fig.suptitle("Difference Between Actual and Threshold - 220210", fontsize=18)

sns.stripplot(data=ALARM_VIOLATIONS_220210,x='THRESHOLD_TYPE', y='DIF_ACTUAL_TH', ax=axs[0])
#axs[0].set_title("Difference Between Actual and Threshold - 220045 Scatter Plot",fontsize=14)
axs[0].set_ylabel("Difference Between Actual and Threshold",fontsize=14)
axs[0].set_xlabel("Threshold Type",fontsize=14)

sns.boxplot(data=ALARM_VIOLATIONS_220210,x='THRESHOLD_TYPE', y='DIF_ACTUAL_TH', ax=axs[1])
#axs[1].set_title("Difference Between Actual and Threshold - 220045 Boxplot")
axs[1].set_ylabel("Difference Between Actual and Threshold", fontsize=14)
axs[1].set_xlabel("Threshold Type",fontsize=14)

#sns.histplot(data=ALARM_VIOLATIONS_220210, x='DIF_ACTUAL_TH', ax=axs[2])
#axs[2].set_title("HR_violations_clean histogram")
#axs[2].set_xlabel("HR_violations_clean VALUENUM")

plt.show(fig)

### 220277 - O2 saturation pulseoxymetry

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
ALARM_VIOLATIONS_220277 = ALARM_VIOLATIONS[(ALARM_VIOLATIONS['ITEMID'] == '220277')]

fig, axs = plt.subplots(1, 2, figsize=(25, 5))
fig.suptitle("Difference Between Actual and Threshold - 220277", fontsize=18)

sns.stripplot(data=ALARM_VIOLATIONS_220277,x='THRESHOLD_TYPE', y='DIF_ACTUAL_TH', ax=axs[0])
#axs[0].set_title("Difference Between Actual and Threshold - 220045 Scatter Plot",fontsize=14)
axs[0].set_ylabel("Difference Between Actual and Threshold",fontsize=14)
axs[0].set_xlabel("Threshold Type",fontsize=14)

sns.boxplot(data=ALARM_VIOLATIONS_220277,x='THRESHOLD_TYPE', y='DIF_ACTUAL_TH', ax=axs[1])
#axs[1].set_title("Difference Between Actual and Threshold - 220045 Boxplot")
axs[1].set_ylabel("Difference Between Actual and Threshold", fontsize=14)
axs[1].set_xlabel("Threshold Type",fontsize=14)

#sns.histplot(data=ALARM_VIOLATIONS_220277, x='DIF_ACTUAL_TH', ax=axs[2])
#axs[2].set_title("HR_violations_clean histogram")
#axs[2].set_xlabel("HR_violations_clean VALUENUM")

plt.show(fig)

### 224687 - Minute Volume

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
ALARM_VIOLATIONS_224687 = ALARM_VIOLATIONS[(ALARM_VIOLATIONS['ITEMID'] == '224687')]

fig, axs = plt.subplots(1, 2, figsize=(25, 5))
fig.suptitle("Difference Between Actual and Threshold - 224687", fontsize=18)

sns.stripplot(data=ALARM_VIOLATIONS_224687,x='THRESHOLD_TYPE', y='DIF_ACTUAL_TH', ax=axs[0])
#axs[0].set_title("Difference Between Actual and Threshold - 220045 Scatter Plot",fontsize=14)
axs[0].set_ylabel("Difference Between Actual and Threshold",fontsize=14)
axs[0].set_xlabel("Threshold Type",fontsize=14)

sns.boxplot(data=ALARM_VIOLATIONS_224687,x='THRESHOLD_TYPE', y='DIF_ACTUAL_TH', ax=axs[1])
#axs[1].set_title("Difference Between Actual and Threshold - 220045 Boxplot")
axs[1].set_ylabel("Difference Between Actual and Threshold", fontsize=14)
axs[1].set_xlabel("Threshold Type",fontsize=14)

#sns.histplot(data=ALARM_VIOLATIONS_224687, x='DIF_ACTUAL_TH', ax=axs[2])
#axs[2].set_title("HR_violations_clean histogram")
#axs[2].set_xlabel("HR_violations_clean VALUENUM")

plt.show(fig)

In [ ]:
## Fragen die wir auf CHARTEVENTS beantworten müssen
# Kann man anhand der Uhrzeit etwas ableiten? ZB nachts treten mehr alarms auf als tags?
# Nach wie viel minuten wird im durchschnitt nach einstellung eines treshholds ein alarm ausgelöst, stratified nach ITEM ID?
# These: "Je häugifer pro Zeiteinheit die nurse den Treshhold anpasst, desto eher gibt es eine violation"
# Um Violation zu predicten, benötigen wir ein erweitertes Datenset (siehe) alarm_violations_extended_by_normal_measurements.png
#Wir bräuchten auch den patienten wieder drin um die patienten demographics zu erhalten
